Just a reminder:
RCFD2107: Obligations (other than securities and leases) of states and political subdivisions in the
U.S (Consolidated Bank). Dollar amounts in thousands.
RCON2107: Obligations (other than securities and leases) of states and political subdivisions in the
U.S (Domestic Offices). Dollar amounts in thousands.

In [2]:
import pandas as pd
import numpy as np
import zipfile
import io
import os
import gc

directory = os.fsencode('data')
muni_banks_table = pd.DataFrame()
quarter = {'0331': 'Q1', '0630': 'Q2', '0930': 'Q3', '1231': 'Q4'}

for file in os.listdir(directory):
    # prep the zipfiles to open
    filename = os.fsdecode(file)
    zipf = zipfile.ZipFile('data/'+filename)
    
    # find some designation to find the right file:
    words = filename.split()
    call_report_date = words[-1].replace('.zip', '')
    
    # find the bank designation
    bank_filename = 'FFIEC CDR Call Bulk POR {}.txt'.format(call_report_date)
    bank_file = pd.read_csv(zipf.open(bank_filename), sep='\t').set_index('IDRSSD')

    # grab the call schedule
    securities_filename = 'FFIEC CDR Call Schedule RCCI {}.txt'.format(call_report_date)
    securities_file = pd.read_csv(zipf.open(securities_filename), sep='\t', header=[0], skiprows=[1]).set_index('IDRSSD')

    # just suss out the muni data
    muni_loans = securities_file['RCFD2107'].dropna().to_frame()
    muni_loans['CallDate'] = call_report_date[-4:]+quarter[call_report_date[0:4]]
    
    # merge the tables together, append to dataframe
    muni_bank = pd.merge(muni_loans, bank_file, how='left', left_index=True, right_index=True)
    if muni_banks_table.empty:
        muni_banks_table = muni_bank
    else:
        muni_banks_table = pd.concat([muni_banks_table, muni_bank])
        
    # garbage clean up
    del [[bank_file, securities_file, muni_loans, muni_bank]]
    gc.collect()
    bank_file = pd.DataFrame()
    securities_file = pd.DataFrame()
    muni_loans = pd.DataFrame()
    muni_bank = pd.DataFrame()

In [4]:
# let's look at the sums
muni_banks_table[muni_banks_table['RCFD2107'] != 0]
muni_total = muni_banks_table.groupby(['CallDate', 'Financial Institution Name'])['RCFD2107'].sum().unstack(level=-2).fillna(0)
temp = muni_total.transpose()
temp['ALL BANKS'] = temp.sum(axis=1)
muni_total = temp.transpose()
muni_total = muni_total/1000
muni_total['sum'] = muni_total.sum(axis=1)
muni_total = muni_total[muni_total['sum'] != 0]
muni_total.drop('sum', axis=1, inplace=True)

total_only = muni_total[muni_total.index=='ALL BANKS'].copy().transpose()
# let's drop the total for now DELETE THIS LINE IF YOU WANT TO KEEP IT
muni_total = muni_total.drop('ALL BANKS', axis=0)

Then taking all that work and converting it into an easy to read csv for heroku to fly with.

In [5]:
muni_total.to_csv('munidata.csv')
total_only.to_csv('munitotal.csv')